In [4]:
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

In [5]:
wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)
col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [6]:
for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,576
2,8,4,20,15,445,0.99,0.99,398.1,7962,398
3,9,1,5,20,355,1.03,0.8,396.75,7935,472
4,8,5,10,15,350,0.9,0.91,390.25,7805,468
5,8,1,10,16,345,0.9,0.91,387.9,7758,465
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,272
68,17,1,23,22,565,1.03,0.8,289.55,5791,334
69,19,5,30,10,475,0.95,0.92,289.1,5782,311
70,6,5,15,5,250,1,0.92,288.9,5778,308


In [15]:
array_aePerulangan = []
for index, row in col1.iterrows():
    print()
    print("____ DATA ",index,"____")

    #Setting Parameter

    c1 = 2
    c2 = 2
    r1 = (0, 1)
    r2 = (0, 1)
    iteration_max = 10 #40
    particles = 10 #70
    fitness_value = 200
    productivity = 20
    inertia_max = 0.9
    inertia_min = 0.4
    inertia = 0
    temp = 0
    ae = 0
    particles_position = [0]

    #perhitungan UCP
    def ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf):
        ucSimple = xSimple * simpleUC
        ucAverage = xAverage * averageUC
        ucComplex = xComplex * complexUC
    
        uucw = ucSimple + ucAverage + ucComplex
        uucp = uaw + uucw
        return uucp * tcf * ecf

    #Generate random Simple Use Case Complexity Weight Parameter
    def randomSimpleUCW(lower_bound=5, upper_bound=7.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Average Use Case Complexity Weight Parameter 
    def randomAverageUCW(lower_bound=7.5, upper_bound=12.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Complex Use Case Complexity Weight Parameter
    def randomComplexUCW(lower_bound=12.5, upper_bound=15, n_particles=10):
        #inisialisasi posisi awal
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]
    
    swarm_size = 5 #70 
    simpleUC = col1.loc[index,"S"]
    averageUC = col1.loc[index,"A"]
    complexUC = col1.loc[index,"C"]
    estimated_effort = 0
    uaw = col1.loc[index,"UAW"]
    uucw = col1.loc[index,"UUCW"]
    tcf = col1.loc[index,"TCF"]
    ecf = col1.loc[index,"ECF"]
    real_effort = col1.loc[index,"Real_Effort"]
    array_aeParticle = []
    ae_minimum = 5000
    particle = []
    
    for i in range(iteration_max):
        inertia = inertia_max - ((inertia_max-inertia_min)*i/iteration_max)
        print('Perulangan ',i)
        print('inertia :', inertia)
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        print('r1 :', r1)
        print('r2 :', r2)
        print()
        
        if i == 0:
            for j in range(swarm_size):    
                xSimple = randomSimpleUCW()
                xAverage = randomAverageUCW()
                xComplex = randomComplexUCW()
                case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                estimated_effort = case * 20
        
                #Selisih estimasi effort
                ae = abs(estimated_effort - real_effort)
            
                #Menyimpan ae minimum dan letaknya
                if ae_minimum > ae:
                    ae_minimum = ae
                    letak_i = i
                    letak_j = j
                    xSimpleMin = xSimple
                    xAverageMin = xAverage
                    xComplexMin = xComplex
            
                #Memasukkan selisih
                #array_aeParticle.append(ae)
                #aeParticle_min = np.min(array_aeParticle)
                #index_of_minimum = np.where(array_aeParticle == aeParticle_min)
        
                #menjumlah 10 angka selisih
                temp = np.array(temp+ae)
         
                #print('Partikel ke', j,':',estimated_effort)
                #print('ucp   ', case)
                #print('ae ', j, ae)
                #print('S     ', xSimple)
                #print('A     ', xAverage)
                #print('C     ', xComplex)
                #print()
            
                #print('estimated effort', estimated_effort)
            
                if j == 0:
                    pbest_simple = xSimple
                    pbest_average = xAverage
                    pbest_complex = xComplex
                
                    vSimple = random.uniform(0, 1)
                    vAverage = random.uniform(0, 1)
                    vComplex = random.uniform(0, 1)
                
                    xSimple = pbest_simple
                    xAveragepbest_average = pbest_average
                    xComplex = pbest_complex
                
                    xAverage = randomAverageUCW()
                    xComplex = randomComplexUCW()
                    case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                    estimated_effort = case * 20
                    
                    #Selisih estimasi effort
                    ae = abs(estimated_effort - real_effort)
                    
                    #Simpan nilai ke dalam Array
                    particle = [[xSimple, xAverage, xComplex, vSimple, vAverage, vComplex, ae]]
                    print(particle)
                      
            
                #Hitung velocity
                #vSimple = inertia * vSimple + (c1 * r1) * (pbest_simple - xSimple) + (c2* r2) * 
            
            #Menyimpan ae minimum dan letaknya
            if ae_minimum > ae:
                ae_minimum = ae
                letak_i = i
                letak_j = j
                xSimpleMin = xSimple
                xAverageMin = xAverage
                xComplexMin = xComplex
            
            #Memasukkan selisih
            #array_aeParticle.append(ae)
            #aeParticle_min = np.min(array_aeParticle)
            #index_of_minimum = np.where(array_aeParticle == aeParticle_min)
        
            #menjumlah 10 angka selisih
            #temp = np.array(temp+ae)
         
            #print('Partikel ke', j,':',estimated_effort)
            #print('ucp   ', case)
            #print('ae ', j, ae)
            #print('S     ', xSimple)
            #print('A     ', xAverage)
            #print('C     ', xComplex)
            #print()
            
            #print('estimated effort', estimated_effort)
            
        #print('ae terkecil : ',np.min(aeParticle_min))
        #print('letak ae : ',index_of_minimum)
        #print('pada perulangan ke :', aeParticle_min.index(min(aeParticle_min)))
        #mae = temp/swarm_size
        #print('MAE :', mae)
        print()
        #array_aePerulangan.append(aeParticle_min)
    print('ae_minimum', ae_minimum)
    print('i', letak_i)
    print('j', letak_j)
    print('S Min', xSimpleMin)
    print('A Min', xAverageMin)
    print('C Min', xComplexMin)
    #print('ae terkecil keseluruhan : ',np.min(array_aePerulangan))
    #print('letak ae : ',index_of_minimum)


____ DATA  1 ____
Perulangan  0
inertia : 0.9
r1 : 0.583607655184009
r2 : 0.8846069929523589

[[7.400431200136614, 9.731864417257595, 13.280326386136846, 0.10731057244425646, 0.905613054603572, 0.18402792418892422, 3208.2052615325783]]

Perulangan  1
inertia : 0.85
r1 : 0.15564836874061794
r2 : 0.13819607497592046


Perulangan  2
inertia : 0.8
r1 : 0.6657472657159414
r2 : 0.32651877861006606


Perulangan  3
inertia : 0.75
r1 : 0.5078211940918932
r2 : 0.016047021232681757


Perulangan  4
inertia : 0.7
r1 : 0.7472293847877306
r2 : 0.6718469370777005


Perulangan  5
inertia : 0.65
r1 : 0.5676000572866275
r2 : 0.8010564546698207


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.662561554446108
r2 : 0.6015238228304784


Perulangan  7
inertia : 0.55
r1 : 0.12781689250921158
r2 : 0.01418592656306239


Perulangan  8
inertia : 0.5
r1 : 0.81901780484649
r2 : 0.9429587741138721


Perulangan  9
inertia : 0.45
r1 : 0.700924309824376
r2 : 0.18483198613575702


ae_minimum 2713.2656182112205
i 0
j 

inertia : 0.65
r1 : 0.8633194243120499
r2 : 0.3558036885448762


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.4344634406810014
r2 : 0.85995996361127


Perulangan  7
inertia : 0.55
r1 : 0.13842098996154595
r2 : 0.7236712172216958


Perulangan  8
inertia : 0.5
r1 : 0.17520067008259532
r2 : 0.8828658606902431


Perulangan  9
inertia : 0.45
r1 : 0.03849363776102077
r2 : 0.9334694052918817


ae_minimum 2186.29584472442
i 0
j 1
S Min 6.579820324851998
A Min 10.767186176856843
C Min 14.668588698841724

____ DATA  13 ____
Perulangan  0
inertia : 0.9
r1 : 0.33588088991009524
r2 : 0.5072620320227216

[[6.591056250214352, 8.599948679127701, 13.904146919962681, 0.5577197954250717, 0.2947380026693557, 0.4125172282877593, 2498.530670954462]]

Perulangan  1
inertia : 0.85
r1 : 0.9730007306945452
r2 : 0.43216483636262426


Perulangan  2
inertia : 0.8
r1 : 0.09592178877479085
r2 : 0.9213838663900317


Perulangan  3
inertia : 0.75
r1 : 0.5296546729852036
r2 : 0.9375230710687964


Perulangan  4
iner

inertia : 0.5
r1 : 0.102910138140753
r2 : 0.6047721588494274


Perulangan  9
inertia : 0.45
r1 : 0.5239801726541133
r2 : 0.850280929824896


ae_minimum 1405.5800793560975
i 0
j 2
S Min 5.298495097386728
A Min 11.283068243344447
C Min 14.880657901181982

____ DATA  29 ____
Perulangan  0
inertia : 0.9
r1 : 0.09504155219742061
r2 : 0.21736417905020622

[[5.714546525082136, 8.59347286214996, 13.307454993708758, 0.23381830130919623, 0.04877223905082528, 0.9506877479633548, 123.03974644278605]]

Perulangan  1
inertia : 0.85
r1 : 0.7788846772941225
r2 : 0.8155482719856599


Perulangan  2
inertia : 0.8
r1 : 0.19185037165667929
r2 : 0.7856910924454925


Perulangan  3
inertia : 0.75
r1 : 0.8891763640374198
r2 : 0.7109657465421185


Perulangan  4
inertia : 0.7
r1 : 0.5678857082867165
r2 : 0.4163494558331229


Perulangan  5
inertia : 0.65
r1 : 0.6970540911730894
r2 : 0.45013394158995623


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.0681462259596094
r2 : 0.14111454996299166


Perulangan  7
in

j 1
S Min 5.277067643878223
A Min 11.502311503920705
C Min 13.990102358872633

____ DATA  41 ____
Perulangan  0
inertia : 0.9
r1 : 0.5692653229646638
r2 : 0.2800049443593873

[[5.659150614567905, 9.255279439993666, 14.848875220923228, 0.9593971264652893, 0.5871245311120803, 0.7517245063331892, 4021.7845984056953]]

Perulangan  1
inertia : 0.85
r1 : 0.605477753034516
r2 : 0.9236830061604266


Perulangan  2
inertia : 0.8
r1 : 0.8148646876449606
r2 : 0.9337544212767203


Perulangan  3
inertia : 0.75
r1 : 0.7834875368489957
r2 : 0.33258759917390024


Perulangan  4
inertia : 0.7
r1 : 0.1818596661604478
r2 : 0.24592462374629154


Perulangan  5
inertia : 0.65
r1 : 0.3708689793011929
r2 : 0.5173877269595369


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.2624499592445565
r2 : 0.4134164244407328


Perulangan  7
inertia : 0.55
r1 : 0.9023550440924508
r2 : 0.21785491211032837


Perulangan  8
inertia : 0.5
r1 : 0.09171165819342497
r2 : 0.9918591749767722


Perulangan  9
inertia : 0.45
r1 : 0.9

Perulangan  4
inertia : 0.7
r1 : 0.0988989493365543
r2 : 0.7028775397519006


Perulangan  5
inertia : 0.65
r1 : 0.2220803825023513
r2 : 0.6083753154527447


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.8282266442272185
r2 : 0.09460559556240089


Perulangan  7
inertia : 0.55
r1 : 0.9303468036168309
r2 : 0.11866008262038463


Perulangan  8
inertia : 0.5
r1 : 0.0008278419973472007
r2 : 0.934163458649592


Perulangan  9
inertia : 0.45
r1 : 0.8590924783131828
r2 : 0.8195154735719273


ae_minimum 664.4471738642305
i 0
j 1
S Min 6.29506208261541
A Min 11.125210742908543
C Min 14.3810865053472

____ DATA  58 ____
Perulangan  0
inertia : 0.9
r1 : 0.006137832847437963
r2 : 0.5275888414271074

[[6.094765977332442, 7.939928212657235, 13.20264266013995, 0.22258279407329262, 0.8523928794898726, 0.1288493133648373, 606.5751626566216]]

Perulangan  1
inertia : 0.85
r1 : 0.7317049480490553
r2 : 0.41122025896638936


Perulangan  2
inertia : 0.8
r1 : 0.04091988381130207
r2 : 0.20447142910083704


Pe

Perulangan  5
inertia : 0.65
r1 : 0.18102006429137285
r2 : 0.9415551248643576


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.1320109004671064
r2 : 0.9492059199745434


Perulangan  7
inertia : 0.55
r1 : 0.2669249137226103
r2 : 0.9458884936668189


Perulangan  8
inertia : 0.5
r1 : 0.734137799355793
r2 : 0.6710206397745115


Perulangan  9
inertia : 0.45
r1 : 0.8134326685612261
r2 : 0.34762123689932123


ae_minimum 2276.416417471659
i 0
j 1
S Min 5.764613479215155
A Min 7.872294299187258
C Min 12.986377857927662

____ DATA  69 ____
Perulangan  0
inertia : 0.9
r1 : 0.17832268825500486
r2 : 0.9974341021718801

[[7.394944410245782, 7.883590418686737, 13.24063521031367, 0.9405539073352841, 0.35886875896238735, 0.5624105960273178, 1645.0559917776372]]

Perulangan  1
inertia : 0.85
r1 : 0.537325992356218
r2 : 0.8339620201596458


Perulangan  2
inertia : 0.8
r1 : 0.590778328132815
r2 : 0.4705075245839003


Perulangan  3
inertia : 0.75
r1 : 0.9589108006426359
r2 : 0.368745294204484


Perulang